In [1]:
import pandas as pd
import pickle
import requests
import json

from api_keys import prodURL, prodKEY, untappd_ID, untappd_SECRET, untappd_URL  # Local constants imports

#### First a few utilities that may be needed

In [2]:
# Method for searching breweryDB for an untappd beername
def get_bdb_beer_by_untappd_name(beername):
    method = 'beers/'
    params = {'key': prodKEY, 'withSocialAccounts': 'Y', 'withIngredients': 'Y', 'withBreweries': 'Y',
              'name': beername}
    response = requests.get(prodURL + method, params=params)
    return response

#  Same method but with the additional specification of an abv in order to disambiguate results
def get_bdb_beer_by_untappd_name_and_abv(name, abv):
    method = 'beers/'
    params = {'key': prodKEY, 'withSocialAccounts': 'Y', 'withIngredients': 'Y', 'withBreweries': 'Y',
             'abv': str(abv), 'name': name}
    response = requests.get(prodURL + method, params=params)
    return response

#  If attempting to get ingredients of a beer, after having learned its ID on bDB
def get_ingredients(bdb_ID):
    method = 'beer/' + str(bdb_ID) + '/ingredients'
    params = {'key': prodKEY}
    response = requests.get(prodURL + method, params=params)
    return response

### Resetting the stage here:  We've gotten all the IPA's from BreweryDB's API by querying its Beer-By-Style endpoint for styles 26, 30, 31, 171, 172, and 173.  We want User Checkins from Untappd for those IPA's, but have no way to look them up on Untappd yet, other than by their name, which will take too long for current purposes.

Instead, we'll query Untappd by searching for the names of the brewers of those IPA's, and dragging onboard whatever we catch.  Start by getting the breweries' names.

In [7]:
# Open a pickled pandas DataFrame
style_30_df = pd.read_pickle('capstone_1/brewkettle/style_30_df.pkl')

# quick check on the columns we have
style_30_df.columns

Index(['index', 'abv', 'available', 'availableId', 'beerVariation',
       'beerVariationId', 'breweries', 'createDate', 'description',
       'foodPairings', 'glass', 'glasswareId', 'ibu', 'id', 'ingredients',
       'isOrganic', 'isRetired', 'labels', 'name', 'nameDisplay',
       'originalGravity', 'servingTemperature', 'servingTemperatureDisplay',
       'socialAccounts', 'srm', 'srmId', 'status', 'statusDisplay', 'style',
       'styleId', 'updateDate', 'year'],
      dtype='object')

In [8]:
# We want the breweries.  What's the format there?
style_30_df.loc[:3, 'breweries']

0    [{'id': 'Xnf2WT', 'name': 'Dust Bowl Brewing C...
1    [{'id': 'H8jawh', 'name': 'Working Man Brewing...
2    [{'id': 'X38nDm', 'name': 'Three Notch'd Brewi...
3    [{'id': 'FfMi8U', 'name': 'Little Machine Beer...
Name: breweries, dtype: object

In [10]:
# So we want the values of the 'name' key, and we're only using the first name when there are multiple brewers
style_30_brewers = set([brewer[0]['name'] for brewer in style_30_df['breweries']])
len(style_30_brewers)

3632

In [11]:
style_31_df = pd.read_pickle('capstone_1/brewkettle/style_31_df.pkl')
style_31_df.columns

Index(['currentPage', 'data', 'numberOfPages', 'status', 'totalResults'], dtype='object')

#### Styles 31, 26, 171, 172, and 173 were pickled in a different structure than Style 30. The 'data' column holds all the dictionaries that had their own rows in style_30_df, so we need to unpack an extra dimension.

In [16]:
style_31_brewers = set([beer['breweries'][0]['name'] for row in range(style_31_df.shape[0])
                      for beer in style_31_df['data'][row]])
len(style_31_brewers)

1864

In [17]:
# see how many brewers are in the other 4 styles
for sty in [26, 171, 172, 173]:
    df = pd.read_pickle('capstone_1/brewkettle/style_' + str(sty) + '_df.pkl')
    brewers = set([beer['breweries'][0]['name'] for row in range(df.shape[0])
                      for beer in df['data'][row]])
    print(f'style_{sty}_df has {len(brewers)} unique brewers.')

style_26_df has 104 unique brewers.
style_171_df has 57 unique brewers.
style_172_df has 443 unique brewers.
style_173_df has 120 unique brewers.


So as expected, the "American IPA" category has the bulk of the brewers:  3632

And there will be overlap in these numbers:

In [18]:
# How many of the 1864 brewers in the second largest group, style 31, are not in the largest group, style 30 ?
len(style_31_brewers.difference(style_30_brewers))

312

In [19]:
# Use a brewery generator to search Untappd
search_gen = (b for b in style_30_brewers)
# Track searched terms, for progress indication and to avoid duplicated efforts
searched = set()

In [20]:
def searchBeers(beersearch_params):
    method_endpoint = '/search/beer/'
    query = untappd_URL + method_endpoint
    response = requests.get(query, beersearch_params)
    if response:  # response==True for codes 200-400
        remaining_calls = response.headers['X-Ratelimit-Remaining']
        return remaining_calls, response
    return 'That GET request for "' + beersearch_params + '" failed, with code: ' + str(response.status_code)

In [ ]:
# Compile a list of the search results and periodically save them.
# This is done here as a manual process, due to the variable lengths of the results and
##   the hourly limits.

results = []

while True:   # until hourly limit is reached or every search result for every query has been returned
    try:
        # generate the next search brewery
        search_string = next(search_gen)
        # keep track of what's been searched
        searched.add(search_string)
        # use the offset parameter to control pagination of API calls/results
        params = {'q':search_string, 'limit':50, 'offset':-50, 'client_id':untappd_ID, 'client_secret':untappd_SECRET}
        search_done = False  # sentinel
        remaining_calls = 100  # this could be initialized to anything > 1 (It'll get reset by the response header)
        while not search_done and remaining_calls > 1:
            params['offset'] += 50  # or could set this to 'limit' param
            limit, resp = searchBeers(params)
            remaining_calls = int(limit)
            resp = resp.json()
            if resp['response']['beers']['count'] < 50:  # The final page of results for each brewery
                search_done = True
            results.append(resp)
        print(f'search done:{search_done}, remaining calls = {remaining_calls}')
        if remaining_calls < 2:
            offset = params['offset']
            print(f'Hit the hourly limit. Last search was "{search_string}". Offset was {offset}')
            break
    except StopIteration:
        print('all done.')
        break

Periodically saving the results:

In [ ]:
batch_counter = 23  # (22 was my last one so I incremented to 23 here to avoid accidental overwrite)
################################
#####  CHANGE THE BATCH NUMBER EVERY TIME OR OVERWRITE HARD WORK!!!!
#########################
searchDF = pd.DataFrame(results)
searchDF.to_pickle(f'beer_search_batch_{batch_counter}.pkl')

### Timeout to inspect results

In [28]:
batch_0 = pd.read_pickle('capstone_1/beerSearches/beer_search_batch_0.pkl')

batch_0.shape

(1, 85)

In [30]:
batch_0.iloc[0,0].keys()

dict_keys(['meta', 'notifications', 'response'])

Drill down through the keys to see the nesting structure

In [31]:
batch_0.iloc[0,0]['response'].keys()

dict_keys(['message', 'time_taken', 'brewery_id', 'search_type', 'type_id', 'search_version', 'found', 'offset', 'limit', 'term', 'parsed_term', 'beers', 'homebrew', 'breweries'])

In [32]:
batch_0.iloc[0,0]['response']['beers'].keys()

dict_keys(['count', 'items'])

In [33]:
batch_0.iloc[0,0]['response']['beers']['items'][0].keys()

dict_keys(['checkin_count', 'have_had', 'your_count', 'beer', 'brewery'])

In [45]:
batch_0.iloc[0,0]['response']['beers']['items'][0]['beer']

{'auth_rating': 0,
 'beer_abv': 5.6,
 'beer_description': 'Our most popular beer, Sierra Nevada Pale Ale is a delightful interpretation of a classic style. It has a deep amber color and an exceptionally full-bodied, complex character. Generous quantities of premium Cascade hops give the Pale Ale its fragrant bouquet and spicy flavor. ',
 'beer_ibu': 38,
 'beer_label': 'https://untappd.akamaized.net/site/beer_logos/beer-6284_5418f_sm.jpeg',
 'beer_name': 'Pale Ale',
 'beer_slug': 'sierra-nevada-brewing-co-pale-ale',
 'beer_style': 'Pale Ale - American',
 'bid': 6284,
 'created_at': 'Thu, 07 Oct 2010 10:23:34 +0000',
 'in_production': 1,
 'wish_list': False}

In [35]:
# BreweryDB results were searchable/matchable by <brewery_name beer_name> format, so let's get that for these.
test_beer = batch_0.iloc[0,0]['response']['beers']['items'][0]
(test_beer['brewery']['brewery_name'] + ' ' + test_beer['beer']['beer_name'])

'Sierra Nevada Brewing Co. Pale Ale'

In [36]:
found_names = set()  # collect results here
messy = []   # collect parsing issues here

for col in range(batch_0.shape[1]):
    try:
        for beer in batch_0.iloc[0,col]['response']['beers']['items']:
            name = beer['brewery']['brewery_name'] + ' ' + beer['beer']['beer_name']
            found_names.add(name)
    except:
        print(f'Column {col} does not parse right')
        messy.append(batch_0.iloc[0,col])
        break
                           

In [37]:
len(messy)

0

In [41]:
print(len(found_names))
#found_names     # commenting out in case scrolling isn't set

4122


Looks like that worked well, so let's add on the other 22 batches

In [42]:
# Note that these frames got structured vertically rather than horiz, so we iterate over rows not cols
for i in range(1,23):  
    frame = pd.read_pickle('capstone_1/beerSearches/beer_search_batch_' + str(i) + '.pkl')
    for row in range(frame.shape[0]):
        try:
            for beer in frame.loc[row,'response']['beers']['items']:
                name = beer['brewery']['brewery_name'] + ' ' + beer['beer']['beer_name']
                found_names.add(name)
        except:
            print(f'Batch {i} Row {row} does not parse right')
            messy.append(frame.loc[row,'response'])
            break

In [43]:
len(messy)

0

In [44]:
len(found_names)

99774

### How many of those names (most aren't even IPA's) match the BreweryDB IPA names?

In [57]:
style_30_df.columns

Index(['index', 'abv', 'available', 'availableId', 'beerVariation',
       'beerVariationId', 'breweries', 'createDate', 'description',
       'foodPairings', 'glass', 'glasswareId', 'ibu', 'id', 'ingredients',
       'isOrganic', 'isRetired', 'labels', 'name', 'nameDisplay',
       'originalGravity', 'servingTemperature', 'servingTemperatureDisplay',
       'socialAccounts', 'srm', 'srmId', 'status', 'statusDisplay', 'style',
       'styleId', 'updateDate', 'year'],
      dtype='object')

In [61]:
# style_30_df was the one with the more unpacked structure, so just combine breweries and name columns
style_30_df['fullname'] = style_30_df.apply(lambda beer: beer['breweries'][0]['name'] + ' ' +
                                           beer['name'], axis=1)

In [62]:
style_30_df.loc[:10, 'fullname']

0      Dust Bowl Brewing Company "Galactic Wrath" IPA
1          Working Man Brewing Company "Ignition" IPA
2     Three Notch'd Brewing Company "Roux 40" Red IPA
3                     Little Machine Beer "Sniff" IPA
4                       Four Fathers Brewing, LLC #15
5                Victory Brewing Company #429 Red IPA
6                  Evil Genius Beer Company #Adulting
7                  Ohana Brewing Co #Hashtag Hops IPA
8                 Thomas Hooker Brewing #NOFILTER IPA
9                             Mikkeller #Overtime IPA
10     Three Floyds Brewing Company $600 Lizard Shoes
Name: fullname, dtype: object

In [64]:
# how many dupes wee there?
print(style_30_df.shape[0])
print(len(set(style_30_df['fullname'])))

9650
9619


Now for the other 5 df's

In [65]:
style_31_df.shape

(83, 5)

In [66]:
style_31_df.columns

Index(['currentPage', 'data', 'numberOfPages', 'status', 'totalResults'], dtype='object')

In [73]:
style_31_df.data[0][0].keys()

dict_keys(['id', 'name', 'nameDisplay', 'description', 'abv', 'ibu', 'glasswareId', 'availableId', 'styleId', 'isOrganic', 'isRetired', 'status', 'statusDisplay', 'createDate', 'updateDate', 'glass', 'available', 'style', 'socialAccounts', 'breweries'])

In [74]:
s31_names = set([beer['breweries'][0]['name'] + ' ' + beer['name'] for row in range(style_31_df.shape[0])
                      for beer in style_31_df['data'][row]])

In [75]:
len(s31_names)

4102

In [76]:
# initialize a master set from df's 30 and 31, then add the remaining 4 styles to it
bdb_IPA_nameset = set(style_30_df['fullname']).union(s31_names)
for sty in [26, 171, 172, 173]:
    df = pd.read_pickle('capstone_1/brewkettle/style_' + str(sty) + '_df.pkl')
    fullnames = set([beer['breweries'][0]['name'] + ' ' + beer['name']
                     for row in range(df.shape[0])
                      for beer in df['data'][row]])
    print(f'style_{sty}_df has {len(fullnames)} unique beernames.')
    bdb_IPA_nameset = bdb_IPA_nameset.union(fullnames)
    
len(bdb_IPA_nameset)

style_26_df has 127 unique beernames.
style_171_df has 64 unique beernames.
style_172_df has 659 unique beernames.
style_173_df has 163 unique beernames.


14733

#### so......how many matches?

In [77]:
len(bdb_IPA_nameset.intersection(found_names))

1175

#### ...and how many of the ~100K untappd results are IPA's?

In [88]:
from collections import Counter
styleCounter = Counter()

In [90]:
# batch_0 had the horizontal aspect, so start with it and then add batches 1-22

batch_0_styles = [beer['beer']['beer_style'] for col in range(batch_0.shape[1])
                  for beer in batch_0.iloc[0,col]['response']['beers']['items']]
styleCounter.update(batch_0_styles)

for i in range(1,23):  
    frame = pd.read_pickle('capstone_1/beerSearches/beer_search_batch_' + str(i) + '.pkl')
    steez = [beer['beer']['beer_style'] for row in range(frame.shape[0])
            for beer in frame.loc[row,'response']['beers']['items']]
    styleCounter.update(steez)

In [91]:
styleCounter

Counter({'Adambier': 7,
         'Altbier': 330,
         'American Wild Ale': 884,
         'Australian Sparkling Ale': 21,
         'Barleywine - American': 741,
         'Barleywine - English': 340,
         'Barleywine - Other': 112,
         'Belgian Blonde': 715,
         'Belgian Dubbel': 586,
         'Belgian Quadrupel': 431,
         'Belgian Strong Dark Ale': 512,
         'Belgian Strong Golden Ale': 423,
         'Belgian Tripel': 835,
         'Bière de Champagne / Bière Brut': 34,
         'Bière de Garde': 227,
         'Bière de Mars': 21,
         'Black & Tan': 32,
         'Blonde Ale': 1969,
         'Bock - Doppelbock': 373,
         'Bock - Eisbock (Traditional)': 37,
         'Bock - Hell / Maibock / Lentebock': 220,
         'Bock - Single / Traditional': 250,
         'Bock - Weizenbock': 163,
         'Brett Beer': 14,
         'Brown Ale - American': 1600,
         'Brown Ale - Belgian': 141,
         'Brown Ale - English': 480,
         'Brown Ale - Imperia

In [92]:
sum(styleCounter.values())

102765

In [95]:
sum(styleCounter[s] for s in styleCounter if s.startswith("IPA"))

27416

In [97]:
styleCounter.most_common(5)

[('IPA - American', 12815),
 ('Pale Ale - American', 5762),
 ('IPA - Imperial / Double', 5168),
 ('Farmhouse Ale - Saison', 4776),
 ('IPA - New England', 2972)]

### Getting Checkins for specific beer ID's from Untappd

In [98]:
def searchBeerFeeds(beerFeed_params, bid):
    method_endpoint = '/beer/checkins/' + str(bid)
    query = untappd_URL + method_endpoint
    response = requests.get(query, beerFeed_params)
    if response:  # response==True for codes 200-400
        remaining_calls = response.headers['X-Ratelimit-Remaining']
        return remaining_calls, response
    else: 
        print(f"That GET request for beer {bid} with params={beerFeed_params.items()} \
failed, with code: {response.status_code}")
        print(response.json())
        return 0,0

In [115]:
# Tweak these params to test how the method fails, so it doesn't fail disruptively during a large search
searchBeerFeeds({'min_id':11, 'max_id':0, 'client_id':untappd_ID, 'client_secret':untappd_SECRET}, 179610)

That GET request for beer 179610 with params=dict_items([('min_id', 11), ('max_id', 0), ('client_id', 'FCD54D21A1313DC49DB8E6DA0B3EE321503A1EF1'), ('client_secret', '2615C26DDEA972D062A5E929EDD0D8E3E9160AD2')]) failed, with code: 400
{'meta': {'code': 400, 'error_detail': "Your 'min_id' is too low, please use a valid that is closer to the most recent ID. Your 'min_id' must be greater than 10 days from now.", 'error_type': 'invalid_param', 'developer_friendly': '', 'response_time': {'time': 0.012, 'measure': 'seconds'}}, 'response': []}


(0, 0)

In [117]:
## Since the goal of calling the BeerActivityFeed on the API is not to exhaust some beer list,
#   but rather to gather reviews on every joined beer, this routine should take as input a
#   beer_id ('bid') and a number of calls to make on that bid.  Then store with the results
#   an ID of which checkin to start with next time you run that bid. Store the first checkin ID
#   as well, to not run over the same ground on newer calls in the future.

def brewFeedBatch(bid, builder_dict, olderthan=None, newerthan=None, numCalls=12):
    '''
    This method makes @numCalls queries (default=12 because untappd limits you to 300 most recent
    checkins for a beer, so 300/25perpage calls) to untappd's beer Activity Feed,
    for beer ID @bid, using either @olderthan or @newerthan to let untappd know
    where to start (going back in time from @olderthan, which is untappd's 
    "max_id" param) or end (going from now to @newerthan, which is "min_id").
    
    The results and updated pagination limits are added to the 
    @builder_dict passed in.
    '''
    params = {'max_id': olderthan, 'min_id': newerthan, 'client_id':untappd_ID, 'client_secret':untappd_SECRET}
       
    for i in range(numCalls):
        calls_left, response = searchBeerFeeds(params, bid)
        if not response:
            print('NO RESPONSE')
            return
        resp = response.json()
        # start pagination and rate limit bookkeeping
        try:
            new_oldest = resp['response']['pagination']['max_id']
            params['max_id'] = new_oldest
            builder_dict[bid]['oldestID'] = new_oldest
            builder_dict[bid]['datalist'].append(resp)
            # this happens only first time; mark in this beer's dict the youngest checkin seen
            if builder_dict[bid]['newestID'] is None:
                builder_dict[bid]['newestID'] = resp['response']['checkins']['items'][0]['checkin_id']
        except KeyError as kerr:
            print(f'No {kerr} was included in the response.')
            print(f'There are {calls_left} calls left.')
            print(f'The max_id passed in was {olderthan}.')
            print(f'The beerID was {bid}')
            

    print(f'Completed {i+1} calls. Hourly calls remaining: {calls_left}') 
    


Get the untappd beer ID's from the 1175 name matches to query their feeds

In [123]:
# first batch_0 with the horizontal aspect
matched_IDs = set(beer['beer']['bid'] for col in range(batch_0.shape[1])
                  for beer in batch_0.iloc[0,col]['response']['beers']['items']
                 if beer['brewery']['brewery_name'] + ' ' + beer['beer']['beer_name'] in bdb_IPA_nameset)


In [125]:
# that was too long of a comprehension, so will loop out the next 22 for clarity
for batch in range(1, 23):
    frame = pd.read_pickle(f'capstone_1/beerSearches/beer_search_batch_{batch}.pkl')
    for row in range(frame.shape[0]):   # each row is a batch of results from an API call
        for beer in frame.loc[row,'response']['beers']['items']:     
            if beer['brewery']['brewery_name'] + ' ' + beer['beer']['beer_name'] in bdb_IPA_nameset:
                matched_IDs.add(beer['beer']['bid']) 

In [127]:
len(matched_IDs)
# Looks like there are 1180 beer IDs and 1175 unique '<brewery> <beer>' combos.  Not unusual for name overlaps.

1180

In [129]:
# make a generator for beer ID's to feed into the calls
bID_gen = (b for b in matched_IDs)
# don't re-initialize this unless you want extra work rebuilding its state

In [128]:
# initialize a permanent dict of dicts to contain, for each bID, the pagination info as well as all feeds accrued

beerFeedDicts = {bID: {'newestID':None, 'oldestID':None, 'datalist':[]} for bID in matched_IDs}

#### Manually run this cell and tweak parameters as needed to keep the flow on

In [ ]:
# for managing API rate limits, get a clock
import time

for batch in range(8):  #  100 calls limit / (300 max checkins / 25 checkins per call) = 8 batches/ hour
    beerID = next(bid_gen)
                     # or  (newerthan=beerFeedDicts[beerID]['newestID'])   to fill in future checkins
    brewFeedBatch(beerID, olderthan=beerFeedDicts[beerID]['oldestID'], numCalls=12)
    print(f'Batch {batch} finished at {time.asctime()[11:16]}')

#### When you want to save the results, run the cell below, and then reset the beerFeedDicts for the next batch

In [ ]:
batch = 0
with open(f'capstone_1/beerFeeds/beerFeedDicts_{batch}.pkl', 'wb') as f:
    pickle.dump(beerFeedDicts, f)

### After getting enough Beer Feeds to get you started

Find out which Users are responsible for the highest amount of those Checkins

In [161]:
#********************** THIS SLOWED MY LAPTOP TO A CRAWL AND TOOK 9 MINUTES FOR 210K CHECKINS
from glob import glob
# depending on what files you saved in the above Beer Feed Getter routine:
beer_feed_files = glob('capstone_1/beerFeeds/beerFeedDicts*')
userCounter = Counter()
styles = Counter()

for file in beer_feed_files:
    with open(file, 'rb') as f:
        beerdicts = pickle.load(f)
    # There is a ton of data in these beerdicts, so generate and parse in bits
    beer_feed_dict_gen = (datachunk for val in beerdicts.values() for datachunk in val['datalist'])

    while True:
        try:
            chunk = next(beer_feed_dict_gen)
            userCounter.update([item['user']['user_name'] 
                               for item in chunk['response']['checkins']['items']])
            styles.update([item['beer']['beer_style']
                          for item in chunk['response']['checkins']['items']])
        except StopIteration:
            break
print(len(userCounter.keys()))

86041


In [159]:
# This is the order we'll search User Feeds in, for the balance of the data gathering
userCounter.most_common(20)

[('jjstark24', 128),
 ('Rally', 88),
 ('collinmcclendon', 86),
 ('Stark_Stack', 83),
 ('kernel', 66),
 ('swoopjones', 52),
 ('taciturnity7', 52),
 ('innajunglestyle', 50),
 ('NYbeerogre', 46),
 ('Chudini', 44),
 ('kansasblonde', 44),
 ('Ryan_wollert', 44),
 ('Straymon', 39),
 ('BVery', 38),
 ('Jc1327', 38),
 ('Beerhopperken', 38),
 ('Sjones132', 37),
 ('Holydiverr33', 36),
 ('PoundingIPAs', 34),
 ('punkatz13', 33)]

In [160]:
# How many checkins did we get at this point, now that we're going to switch to querying User Feeds?
sum(userCounter.values())

209722

In [162]:
# And how many of those are IPA's ?
sum(styles[s] for s in styles if s.startswith('IPA'))

200323

In [165]:
styles.most_common()

[('IPA - American', 145736),
 ('IPA - New England', 14504),
 ('IPA - Imperial / Double', 9196),
 ('IPA - Brut', 8184),
 ('IPA - Black / Cascadian Dark Ale', 5165),
 ('Pale Ale - American', 3226),
 ('IPA - Rye', 2750),
 ('IPA - White', 2750),
 ('IPA - Session / India Session Ale', 2200),
 ('IPA - Belgian', 2150),
 ('IPA - English', 1925),
 ('IPA - Red', 1913),
 ('Rye Beer', 1650),
 ('IPA - International', 1375),
 ('IPA - Sour', 1100),
 ('IPA - Imperial / Double New England', 1100),
 ('Spiced / Herbed Beer', 1050),
 ('Red Ale - American Amber / Red', 550),
 ('Pale Ale - Belgian', 550),
 ('Fruit Beer', 550),
 ('Farmhouse Ale - Saison', 514),
 ('Blonde Ale', 484),
 ('Winter Ale', 275),
 ('Freeze-distilled Beer', 275),
 ('Lager - IPL (India Pale Lager)', 275),
 ('IPA - Imperial / Double Milkshake', 275)]